In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/gonzalo/Downloads/nema-floodmapper-2022.json"

In [2]:
from ml4floods.data import utils
import geopandas as gpd
from georeader.readers import ee_query, scihubcopernicus_query
import folium

aois = utils.read_geojson_from_gcp("gs://ml4floods_nema/0_DEV/1_Staging/operational/NEMA002/aois.geojson")
area_of_interest = aois.geometry.unary_union
df_aoi = gpd.GeoDataFrame(geometry=[area_of_interest],crs="EPSG:4326")
m = aois.explore()
df_aoi.explore(m=m,color="red")

In [3]:
aois.duplicated().any()

False

## Set up the dates to query

In [4]:
from datetime import datetime, timezone, timedelta
import ee
import geopandas as gpd
import pandas as pd
import folium
from zoneinfo import ZoneInfo

tz = ZoneInfo("Australia/Sydney")

date_event = datetime.strptime("2022-10-11","%Y-%m-%d").replace(tzinfo=tz)

period_pre_flood_start = datetime.strptime("2022-09-05","%Y-%m-%d").replace(tzinfo=tz)
period_pre_flood_end = datetime.strptime("2022-09-22","%Y-%m-%d").replace(tzinfo=tz)

period_flood_start = datetime.strptime("2022-10-11","%Y-%m-%d").replace(tzinfo=tz)
period_flood_end = period_flood_start + timedelta(days=15)


print(f"Searching pre-flood images between {period_pre_flood_start} and {period_pre_flood_end} and post-flood between: {period_flood_start} and {period_flood_end}")

Searching pre-flood images between 2022-09-05 00:00:00+10:00 and 2022-09-22 00:00:00+10:00 and post-flood between: 2022-10-11 00:00:00+11:00 and 2022-10-26 00:00:00+11:00


## Images available in GEE

In [5]:
%%time

# images_available_gee_s2 = ee_query.query(area_of_interest, period_flood_start, period_flood_end)
# images_available_gee = ee_query.query(area_of_interest, period_flood_start, period_flood_end,producttype="Landsat")
# images_available_gee = pd.concat([images_available_gee_s2, images_available_gee_landsat]).sort_values("utcdatetime")
images_available_gee, collection = ee_query.query(area_of_interest, period_flood_start, period_flood_end,producttype="both", 
                                                  return_collection=True)

assert images_available_gee.shape[0] > 0, "No images found for date and loc"
    
images_available_gee[["overlappercentage","cloudcoverpercentage", "utcdatetime",
                      "localdatetime","solardatetime","solarday","satellite"]]

CPU times: user 2.63 s, sys: 23.3 ms, total: 2.65 s
Wall time: 17.6 s


,overlappercentage,cloudcoverpercentage,utcdatetime,localdatetime,solardatetime,solarday,satellite
title,,,,,,,
LC09_L1TP_092082_20221011_20221011_02_T1,6.406580,97.690000,2022-10-11 00:02:15.716000+00:00,2022-10-11 11:02:15.716000+11:00,2022-10-11 09:49:38.559155+00:00,2022-10-11,LC09
LC09_L1TP_092081_20221011_20221011_02_T1,6.385959,71.040000,2022-10-11 00:01:51.765000+00:00,2022-10-11 11:01:51.765000+11:00,2022-10-11 09:49:14.608155+00:00,2022-10-11,LC09
LC09_L1TP_092085_20221011_20221011_02_T1,5.281227,71.700000,2022-10-11 00:03:27.584000+00:00,2022-10-11 11:03:27.584000+11:00,2022-10-11 09:50:50.427155+00:00,2022-10-11,LC09
LC09_L1TP_092083_20221011_20221011_02_T1,5.139848,58.470000,2022-10-11 00:02:39.670000+00:00,2022-10-11 11:02:39.670000+11:00,2022-10-11 09:50:02.513155+00:00,2022-10-11,LC09
LC09_L1TP_092086_20221011_20221011_02_T1,4.776287,86.040000,2022-10-11 00:03:51.538000+00:00,2022-10-11 11:03:51.538000+11:00,2022-10-11 09:51:14.381155+00:00,2022-10-11,LC09
...,...,...,...,...,...,...,...
S2B_MSIL1C_20221024T001109_N0400_R073_T55JDG_20221024T011240,0.073304,44.354289,2022-10-24 00:15:25.880000+00:00,2022-10-24 11:15:25.880000+11:00,2022-10-24 10:02:48.723155+00:00,2022-10-24,S2B
S2B_MSIL1C_20221024T001109_N0400_R073_T55HBU_20221024T011240,0.064018,99.981131,2022-10-24 00:17:29.053000+00:00,2022-10-24 11:17:29.053000+11:00,2022-10-24 10:04:51.896155+00:00,2022-10-24,S2B
S2B_MSIL1C_20221024T001109_N0400_R073_T55GEP_20221024T011240,0.041610,99.763812,2022-10-24 00:18:21.592000+00:00,2022-10-24 11:18:21.592000+11:00,2022-10-24 10:05:44.435155+00:00,2022-10-24,S2B


In [12]:
# pd.options.display.max_rows = 120
# images_available_gee[(images_available_gee["solarday"] >= "2022-10-12") & (images_available_gee["solarday"] <= "2022-10-14")][["overlappercentage","cloudcoverpercentage", "utcdatetime",
#                       "localdatetime","solardatetime","solarday","satellite"]]

In [6]:
images_available_gee["localdatetime_str"] = images_available_gee["localdatetime"].dt.strftime("%Y-%m-%d %H:%M:%S")

stuffshow = ["geometry","overlappercentage","cloudcoverpercentage", "localdatetime_str","solarday","satellite"]

colors = ["#ff7777", "#fffa69", "#8fff84", "#52adf1", "#ff6ac2","#1b6d52", "#fce5cd","#705334"]
m=df_aoi.explore(color="red", name="AoI")

for i, ((day,satellite), images_day) in enumerate(images_available_gee.groupby(["solarday","satellite"])):
    m = images_day[stuffshow].explore(m=m,name=f"{satellite}: {day}",color=colors[i % len(colors)])

folium.LayerControl(collapsed=False).add_to(m)
m

## Display images available with geemap

In [7]:
collection.size().getInfo()

551

In [8]:
%%time

import geemap.foliumap as geemap

m = geemap.Map(location=area_of_interest.centroid.coords[0][-1::-1], zoom_start=6)

for (day,satellite), images_day in images_available_gee.groupby(["solarday", "satellite"]):
    
    image_col_day_sat = collection.filter(ee.Filter.inList("title", images_day.index.tolist()))
    
    bands = ["B11","B8","B4"] if satellite.startswith("S2") else ["B6","B5","B4"]
                                          
    m.addLayer(image_col_day_sat, 
                 {"min":0, "max":3000 if satellite.startswith("S2") else 0.3, 
                  "bands":bands},
                 f"{satellite}: {day}", 
                 False)


df_aoi.explore(color="red", name="AoI",m=m)
folium.LayerControl(collapsed=False).add_to(m)
m

CPU times: user 1.42 s, sys: 48.3 ms, total: 1.47 s
Wall time: 45 s


## Spatial Join

This code outputs the images available for each element in the grid.

In [17]:
from georeader.readers import query_utils

aois_images = images_available_gee.reset_index().sjoin(aois, how="inner").reset_index(drop=True)

print(aois_images.shape)
aois_indexed = aois.set_index("name")

indexes_selected = []
for (day, gridid), products_gpd_day in aois_images.groupby(["solarday","name"]):
    area = aois_indexed.loc[gridid, "geometry"]
    idx_pols_selected = query_utils.select_polygons_overlap(products_gpd_day.geometry.tolist(), area)

    indexes_selected.extend(products_gpd_day.iloc[idx_pols_selected].index.tolist())

aois_images = aois_images.loc[indexes_selected].copy()
aois_images = aois_images.reset_index(drop=True)
aois_images.shape

(12152, 15)


(8031, 15)

In [36]:
aois_images.sort_values(["name","localdatetime"]).set_index(["name", "solarday"])[["title","cloudcoverpercentage","satellite", "localdatetime"]]

title  \
name      solarday                                                        
GRID02947 2022-10-15  S2A_MSIL1C_20221015T002711_N0400_R016_T54HVD_2...   
          2022-10-15  S2A_MSIL1C_20221015T002711_N0400_R016_T54HVC_2...   
          2022-10-20  S2B_MSIL1C_20221020T002709_N0400_R016_T54HVD_2...   
          2022-10-20  S2B_MSIL1C_20221020T002709_N0400_R016_T54HVC_2...   
GRID02948 2022-10-15  S2A_MSIL1C_20221015T002711_N0400_R016_T54HVD_2...   
...                                                                 ...   
GRID13672 2022-10-21  S2B_MSIL1C_20221021T000219_N0400_R030_T55GFQ_2...   
GRID13673 2022-10-11  S2B_MSIL1C_20221011T000219_N0400_R030_T55GFQ_2...   
          2022-10-14           LC08_L1TP_089089_20221013_20221014_02_RT   
          2022-10-16  S2A_MSIL1C_20221016T000231_N0400_R030_T55GFQ_2...   
          2022-10-21  S2B_MSIL1C_20221021T000219_N0400_R030_T55GFQ_2...   

                      cloudcoverpercentage satellite  \
name      solarday                                     
GRID02947 2022-10-15             91.954520       S2A   
          2022-10-15             99.475087       S2A   
          2022-10-20              0.000000       S2B   
          2022-10-20             22.693915       S2B   
GRID02948 2022-10-15             91.954520       S2A   
...                                    ...       ...   
GRID13672 2022-10-21             82.982800       S2B   
GRID13673 2022-10-11             74.633432       S2B   
          2022-10-14             70.590000      LC08   
          2022-10-16             54.498731       S2A   
          2022-10-21             82.982800       S2B   

                                        localdatetime  
name      solarday                                     
GRID02947 2022-10-15 2022-10-15 11:37:34.346000+11:00  
          2022-10-15 2022-10-15 11:37:48.699000+11:00  
          2022-10-20 2022-10-20 11:37:29.514000+11:00  
          2022-10-20 2022-10-20 11:37:43.839000+11:00  
GRID02948 2022-10-15 2022-10-15 11:37:34.346000+11:00  
...                                               ...  
GRID13672 2022-10-21 2022-10-21 11:08:12.113000+11:00  
GRID13673 2022-10-11 2022-10-11 11:08:11.131000+11:00  
          2022-10-14 2022-10-14 10:46:36.455000+11:00  
          2022-10-16 2022-10-16 11:08:16.522000+11:00  
          2022-10-21 2022-10-21 11:08:12.113000+11:00  

[8031 rows x 4 columns]

In [13]:
data_GRID02947 = aois_images_indexed.loc["GRID02947"]
data_GRID02947
m = data_GRID02947[["localdatetime_str", "cloudcoverpercentage","geometry"]].explore(name="images")
aois_GRID02947 = aois[aois["name"] == "GRID02947"]
aois_GRID02947.explore(m=m,name="GRID02947",color="red")
m

In [26]:
aois_images.groupby(["name"])[['localdatetime',"cloudcoverpercentage"]].agg(["min","max","count","mean"])

localdatetime                                   \
                                       min                              max   
name                                                                          
GRID02947 2022-10-15 11:37:34.346000+11:00 2022-10-20 11:37:43.839000+11:00   
GRID02948 2022-10-15 11:37:34.346000+11:00 2022-10-20 11:37:29.514000+11:00   
GRID02949 2022-10-15 11:37:34.346000+11:00 2022-10-20 11:37:29.514000+11:00   
GRID02950 2022-10-15 11:37:34.346000+11:00 2022-10-20 11:37:29.514000+11:00   
GRID03093 2022-10-15 11:37:45.180000+11:00 2022-10-20 11:37:40.342000+11:00   
...                                    ...                              ...   
GRID13650 2022-10-11 11:08:15.094000+11:00 2022-10-21 11:08:16.074000+11:00   
GRID13651 2022-10-11 11:08:15.094000+11:00 2022-10-21 11:08:16.074000+11:00   
GRID13670 2022-10-11 11:08:25.276000+11:00 2022-10-21 11:08:26.257000+11:00   
GRID13672 2022-10-11 11:08:11.131000+11:00 2022-10-21 11:08:12.113000+11:00   
GRID13673 2022-10-11 11:08:11.131000+11:00 2022-10-21 11:08:12.113000+11:00   

                                                    cloudcoverpercentage  \
          count                                mean                  min   
name                                                                       
GRID02947     4 2022-10-17 23:37:39.099500032+11:00             0.000000   
GRID02948     2 2022-10-17 23:37:31.929999872+11:00             0.000000   
GRID02949     2 2022-10-17 23:37:31.929999872+11:00             0.000000   
GRID02950     2 2022-10-17 23:37:31.929999872+11:00             0.000000   
GRID03093     2 2022-10-17 23:37:42.760999936+11:00             2.938905   
...         ...                                 ...                  ...   
GRID13650     4 2022-10-15 23:02:52.026749952+11:00            36.895966   
GRID13651     3 2022-10-16 11:08:17.217333248+11:00            36.895966   
GRID13670     7    2022-10-16 07:35:19.556000+11:00            67.003671   
GRID13672     4 2022-10-15 23:02:49.055249920+11:00            54.498731   
GRID13673     4 2022-10-15 23:02:49.055249920+11:00            54.498731   

                                        
                  max count       mean  
name                                    
GRID02947   99.475087     4  53.530881  
GRID02948   91.954520     2  45.977260  
GRID02949   91.954520     2  45.977260  
GRID02950   91.954520     2  45.977260  
GRID03093   93.203948     2  48.071426  
...               ...   ...        ...  
GRID13650   98.915166     4  76.185460  
GRID13651   98.915166     3  78.050614  
GRID13670  100.000000     7  84.962808  
GRID13672   82.982800     4  70.676241  
GRID13673   82.982800     4  70.676241  

[1371 rows x 8 columns]

## Images Available in CopernicusHub

In [8]:
%%time

images_available_copernicushub = scihubcopernicus_query.query(area_of_interest, period_flood_start, period_flood_end)
images_available_copernicushub[["overlappercentage","cloudcoverpercentage", "utcdatetime",
                                "localdatetime","solardatetime","solarday"]]

Querying products:  57%|#####7    | 100/174 [00:00<?, ?product/s]

CPU times: user 1.25 s, sys: 7.59 ms, total: 1.25 s
Wall time: 5.63 s


,overlappercentage,cloudcoverpercentage,utcdatetime,localdatetime,solardatetime,solarday
title,,,,,,
S2B_MSIL1C_20221011T000219_N0400_R030_T56JKL_20221011T011012,2.060235,78.241572,2022-10-11 00:02:19.024000+00:00,2022-10-11 11:02:19.024000+11:00,2022-10-11 09:49:41.867155,2022-10-11
S2B_MSIL1C_20221011T000219_N0400_R030_T56JKN_20221011T011012,1.628636,72.120672,2022-10-11 00:02:19.024000+00:00,2022-10-11 11:02:19.024000+11:00,2022-10-11 09:49:41.867155,2022-10-11
S2B_MSIL1C_20221011T000219_N0400_R030_T55HGB_20221011T011012,1.612859,56.301860,2022-10-11 00:02:19.024000+00:00,2022-10-11 11:02:19.024000+11:00,2022-10-11 09:49:41.867155,2022-10-11
S2B_MSIL1C_20221011T000219_N0400_R030_T55HGC_20221011T011012,1.304508,57.939264,2022-10-11 00:02:19.024000+00:00,2022-10-11 11:02:19.024000+11:00,2022-10-11 09:49:41.867155,2022-10-11
S2B_MSIL1C_20221011T000219_N0400_R030_T55GDN_20221011T011012,1.032979,70.235421,2022-10-11 00:02:19.024000+00:00,2022-10-11 11:02:19.024000+11:00,2022-10-11 09:49:41.867155,2022-10-11
...,...,...,...,...,...,...
S2B_MSIL1C_20221024T001109_N0400_R073_T55JEJ_20221024T011240,0.136813,24.403011,2022-10-24 00:11:09.024000+00:00,2022-10-24 11:11:09.024000+11:00,2022-10-24 09:58:31.867155,2022-10-24
S2B_MSIL1C_20221024T001109_N0400_R073_T55GDM_20221024T013014,0.032426,58.343716,2022-10-24 00:11:09.024000+00:00,2022-10-24 11:11:09.024000+11:00,2022-10-24 09:58:31.867155,2022-10-24
S2B_MSIL1C_20221024T001109_N0400_R073_T55JDG_20221024T011240,0.076612,44.354289,2022-10-24 00:11:09.024000+00:00,2022-10-24 11:11:09.024000+11:00,2022-10-24 09:58:31.867155,2022-10-24


In [9]:
import folium

images_available_copernicushub["localdatetime_str"] = images_available_copernicushub["localdatetime"].dt.strftime("%Y-%m-%d %H:%M:%S")

stuffshow = ["geometry","overlappercentage","cloudcoverpercentage", "localdatetime_str","solarday"]

colors = ["#ff7777", "#fffa69", "#8fff84", "#52adf1", "#ff6ac2","#1b6d52", "#fce5cd","#705334"]
m=df_aoi.explore(color="red", name="AoI")

for i, (day, images_day) in enumerate(images_available_copernicushub.groupby("solarday")):
    m = images_day[stuffshow].explore(m=m,name=f"S2:{day}",color=colors[i % len(colors)])

folium.LayerControl(collapsed=False).add_to(m)

m

## Compare images available in CopernicusHub with images available in GEE

In [12]:
# Sanity check there are no repeated products
images_available_gee_s2 = images_available_gee[images_available_gee.satellite.apply(lambda x: x.startswith("S2"))]
assert images_available_gee_s2.index.unique().shape[0] == images_available_gee_s2.shape[0], "Duplicate images"
assert images_available_copernicushub.index.unique().shape[0] == images_available_copernicushub.shape[0], "Duplicate images"
print(f"S2 images GEE: {images_available_gee_s2.shape[0]}")
print(f"S2 images CopernicusHub: {images_available_copernicushub.shape[0]}")

S2 images GEE: 515
S2 images CopernicusHub: 281


In [13]:
images_only_copernicushub = [img for img in images_available_copernicushub.index if img not in images_available_gee_s2.index]
images_only_copernicushub

['S2B_MSIL1C_20221011T000219_N0400_R030_T55GDM_20221011T011012',
 'S2A_MSIL1C_20221012T002101_N0400_R116_T55JFJ_20221012T012258',
 'S2B_MSIL1C_20221014T001109_N0400_R073_T55GBQ_20221014T081344',
 'S2B_MSIL1C_20221014T001109_N0400_R073_T56JKL_20221014T081344',
 'S2B_MSIL1C_20221014T001109_N0400_R073_T55GBR_20221014T081344',
 'S2B_MSIL1C_20221017T002059_N0400_R116_T55GBQ_20221017T014818',
 'S2B_MSIL1C_20221017T002059_N0400_R116_T55JBH_20221017T012421',
 'S2B_MSIL1C_20221017T002059_N0400_R116_T55HBS_20221017T014818',
 'S2B_MSIL1C_20221021T000219_N0400_R030_T55GDM_20221021T010422',
 'S2A_MSIL1C_20221022T002101_N0400_R116_T55JFJ_20221022T012305',
 'S2B_MSIL1C_20221024T001109_N0400_R073_T55GDM_20221024T013014']

In [14]:
images_only_gee = [img for img in images_available_gee_s2.index if img not in images_available_copernicushub.index]
images_only_gee

['S2B_MSIL1C_20221011T000219_N0400_R030_T55GDP_20221011T011012',
 'S2B_MSIL1C_20221011T000219_N0400_R030_T56HKJ_20221011T011012',
 'S2B_MSIL1C_20221011T000219_N0400_R030_T55GEQ_20221011T011012',
 'S2B_MSIL1C_20221011T000219_N0400_R030_T56JKM_20221011T011012',
 'S2B_MSIL1C_20221011T000219_N0400_R030_T55HGE_20221011T011012',
 'S2B_MSIL1C_20221011T000219_N0400_R030_T55GEP_20221011T011012',
 'S2B_MSIL1C_20221011T000219_N0400_R030_T56HLJ_20221011T011012',
 'S2B_MSIL1C_20221011T000219_N0400_R030_T56JLL_20221011T011012',
 'S2B_MSIL1C_20221011T000219_N0400_R030_T55GDQ_20221011T011012',
 'S2B_MSIL1C_20221011T000219_N0400_R030_T55HGD_20221011T011012',
 'S2B_MSIL1C_20221011T000219_N0400_R030_T55JGF_20221011T011012',
 'S2B_MSIL1C_20221011T000219_N0400_R030_T56JLN_20221011T011012',
 'S2B_MSIL1C_20221011T000219_N0400_R030_T55HFD_20221011T011012',
 'S2B_MSIL1C_20221011T000219_N0400_R030_T55JGG_20221011T011012',
 'S2B_MSIL1C_20221011T000219_N0400_R030_T56HLK_20221011T011012',
 'S2B_MSIL1C_20221011T000

## Are images in GEE available in Google Bucket?

Seems yes!!

In [15]:
%%time

from georeader.readers import S2_SAFE_reader

for product in images_available_gee_s2.index:
    try:
        s2_safe_folder = S2_SAFE_reader.s2_public_bucket_path(product+".SAFE", check_exists=True)
        print(f"Product {s2_safe_folder} found")
    except Exception as e:
        print(f"Product {product} was not found")

Product gs://gcp-public-data-sentinel-2/tiles/55/G/DP/S2B_MSIL1C_20221011T000219_N0400_R030_T55GDP_20221011T011012.SAFE found
Product gs://gcp-public-data-sentinel-2/tiles/56/J/KL/S2B_MSIL1C_20221011T000219_N0400_R030_T56JKL_20221011T011012.SAFE found
Product gs://gcp-public-data-sentinel-2/tiles/56/H/KJ/S2B_MSIL1C_20221011T000219_N0400_R030_T56HKJ_20221011T011012.SAFE found
Product gs://gcp-public-data-sentinel-2/tiles/55/G/EQ/S2B_MSIL1C_20221011T000219_N0400_R030_T55GEQ_20221011T011012.SAFE found
Product gs://gcp-public-data-sentinel-2/tiles/56/J/KM/S2B_MSIL1C_20221011T000219_N0400_R030_T56JKM_20221011T011012.SAFE found
Product gs://gcp-public-data-sentinel-2/tiles/55/H/GE/S2B_MSIL1C_20221011T000219_N0400_R030_T55HGE_20221011T011012.SAFE found
Product gs://gcp-public-data-sentinel-2/tiles/55/H/GB/S2B_MSIL1C_20221011T000219_N0400_R030_T55HGB_20221011T011012.SAFE found
Product gs://gcp-public-data-sentinel-2/tiles/56/J/KN/S2B_MSIL1C_20221011T000219_N0400_R030_T56JKN_20221011T011012.SAF